In [2]:
import pandas as pd
import os
import json

In [6]:
from ax.service.ax_client import AxClient

folder = "/home/sn666/large-scale-data-processing/miniproject/final_results_7jan/530df59e66514d88aa1cc5e6c32fe2bc"

# get the ax_client.json file
ax_client = os.path.join(folder, "ax_client.json")

# read the json file
ax_client = AxClient.load_from_json_file(ax_client)

# print number of pareto-optimal points
print(len(ax_client.get_pareto_optimal_parameters()))

# print hypervolume
print(ax_client.get_hypervolume())

/home/sn666/.conda/envs/lsdp/lib/python3.11/site-packages/ax/storage/json_store/decoder.py:303: AxParameterWarning: `sort_values` is not specified for `ChoiceParameter` "layer_1_size". Defaulting to `True` for parameters of `ParameterType` INT. To override this behavior (or avoid this warning), specify `sort_values` during `ChoiceParameter` construction.
  return _class(
/home/sn666/.conda/envs/lsdp/lib/python3.11/site-packages/ax/storage/json_store/decoder.py:303: AxParameterWarning: `sort_values` is not specified for `ChoiceParameter` "layer_2_size". Defaulting to `True` for parameters of `ParameterType` INT. To override this behavior (or avoid this warning), specify `sort_values` during `ChoiceParameter` construction.
  return _class(
/home/sn666/.conda/envs/lsdp/lib/python3.11/site-packages/ax/storage/json_store/decoder.py:303: AxParameterWarning: `sort_values` is not specified for `ChoiceParameter` "layer_3_size". Defaulting to `True` for parameters of `ParameterType` INT. To over

3
4972.79605827308


In [12]:
results_path = "/home/sn666/large-scale-data-processing/miniproject/final_results"

# remarks_i_want = ["maxaccminparam", "micaccminloss", "minlossminparam", "maxaccminlossminparam"]
relevant_experiments = {}

for result in os.listdir(results_path):
    if os.path.isdir(os.path.join(results_path, result)) == False:
        continue
    # open the args.json file
    with open(os.path.join(results_path, result, "args.json"), "r") as f:
        args = json.load(f)
    
    relevant_experiments[args["remark"]] = result

In [13]:
# sort based on when the experiment was run (when folder was created)
sorted_relevant_experiments = dict(sorted(relevant_experiments.items(), key=lambda item: os.path.getmtime(os.path.join(results_path, item[1]))))

print(sorted_relevant_experiments)

{'8e/moasha-nsga/maxt8gr3red4/maxaccminparam': '823b60cabeaa435aa43676cc44e90b48', '8e/moasha-nsga/maxt8gr3red4/maxaccminloss': '5da00b000e7249a7a316e66b4aa99da4', '8e/moasha-nsga/maxt8gr3red4/minlossminparam': 'c7d86a17515f47ccbf2798a73db27304', '8e/moasha-nsga/maxt8gr3red4/maxaccminlossminparam': 'd9ececa78f114668a617218525e3da25', '8e/fifo/maxaccminparam': '54fe6b2e30c74ca1be8d22332b064803', '8e/fifo/maxaccminloss': '3b3e32426e834c59811009492bd08a2c', '8e/fifo/minlossminparam': '273eece2f4454df488c10494ff06f6c9', '8e/fifo/maxaccminlossminparam': '438d2883810c44948c74bdfdbd6bc9ec', '8e/moasha-nsga/maxt8gr2red6/maxaccminparam': 'fa2efbf633934ad8bff65b917900287e', '8e/moasha-nsga/maxt8gr2red6/maxaccminloss': '19b27df2c7b44dea870d95c9515f1844', '8e/moasha-nsga/maxt8gr2red6/minlossminparam': '430aa53872914dc1a676170748539e11', '8e/moasha-nsga/maxt8gr2red6/maxaccminlossminparam': 'f21b077779414a9baa99c026fb40c576', '8e/moasha-epsnet/maxt8gr1red6/maxaccminparam': '3c1ca484ea324101b3c972411

In [14]:
# What I want to analyse?
# 5th, 6th, 7th, 8th
# 9th, 10th, 11th, 12th

# ========
# useless run
# 0 1 2 3
# ========

# ========
# FIFO run
# 4 5 6 7
# ========

# ========
# MO-ASHA run (grace period = 1)
# 8 9 10 11
# ========

# ========
# MO-ASHA EpsNet run
# 12 13 14 15
# ========

# ========
# MO-ASHA run (grace period = 2)
# 16 17 18 19
# ========

sorted_relevant_experiments = list(sorted_relevant_experiments.values())
sorted_relevant_experiments = sorted_relevant_experiments[16:20]

print(sorted_relevant_experiments)

['0494846591ed4f0ba02c1115aeba0746', '1dec9f6b115f408f8538c45981971da0']


In [15]:
from ax.service.ax_client import AxClient

for experiment in sorted_relevant_experiments:
    print(os.path.join(results_path, "54fe6b2e30c74ca1be8d22332b064803", "tuning_results_df.csv"))
    results_df = pd.read_csv(os.path.join(results_path, experiment, "tuning_results_df.csv"))
    total_time = results_df['time_total_s'].sum()
    # read the args.json file
    with open(os.path.join(results_path, experiment, "args.json"), "r") as f:
        args = json.load(f)
    print(f"Args for experiment: {args}")
    print(f"Experiment: {experiment}, Total time: {total_time / 60}")
    print(f"Experiment: {experiment}, Best accuracy: {results_df['ptl/val_accuracy'].max()}")
    print(f"Experiment: {experiment}, Best loss: {results_df['ptl/val_loss'].min()}")
    
    # load the ax_client
    ax_client = AxClient.load_from_json_file(os.path.join(results_path, experiment, "ax_client.json"), verbose_logging=False)
    print(f"{len(ax_client.get_pareto_optimal_parameters())} points: {ax_client.get_pareto_optimal_parameters()}")
    print(f"Hypervolume: {ax_client.get_hypervolume()}")
    
    # print config of trial with best accuracy, print only keys that start with config/
    best_acc_trial = results_df.loc[results_df['ptl/val_accuracy'].idxmax()]
    config = best_acc_trial.filter(like='config/', axis=0)
    print(f"Experiment: {experiment}, Best accuracy trial config: {config.to_dict()}")

/home/sn666/large-scale-data-processing/miniproject/final_results/54fe6b2e30c74ca1be8d22332b064803/tuning_results_df.csv
Args for experiment: {'num_samples': 25, 'max_num_epochs': 8, 'num_gpus': None, 'objective_1': 'ptl/val_accuracy', 'objective_1_type': 'max', 'objective_1_threshold': 0.9, 'objective_2': 'ptl/model_params', 'objective_2_type': 'min', 'objective_2_threshold': 100000.0, 'objective_3': None, 'objective_3_type': None, 'objective_3_threshold': None, 'max_concurrent': 10, 'use_scheduler': True, 'scheduler_strategy': 'eps_net', 'scheduler_max_t': 8, 'scheduler_grace_period': 2, 'scheduler_reduction_factor': 6, 'accelerator': 'gpu', 'use_scaling_config': True, 'data_path': '/home/sn666/large-scale-data-processing/miniproject/data', 'remark': '8e/moasha-epsnet/maxt8gr2red6/maxaccminparam', 'results_folder': 'final_results'}
Experiment: 0494846591ed4f0ba02c1115aeba0746, Total time: 11.000833404064178
Experiment: 0494846591ed4f0ba02c1115aeba0746, Best accuracy: 0.96484380960464

/home/sn666/.conda/envs/lsdp/lib/python3.11/site-packages/ax/storage/json_store/decoder.py:303: AxParameterWarning: `sort_values` is not specified for `ChoiceParameter` "layer_1_size". Defaulting to `True` for parameters of `ParameterType` INT. To override this behavior (or avoid this warning), specify `sort_values` during `ChoiceParameter` construction.
  return _class(
/home/sn666/.conda/envs/lsdp/lib/python3.11/site-packages/ax/storage/json_store/decoder.py:303: AxParameterWarning: `sort_values` is not specified for `ChoiceParameter` "layer_2_size". Defaulting to `True` for parameters of `ParameterType` INT. To override this behavior (or avoid this warning), specify `sort_values` during `ChoiceParameter` construction.
  return _class(
/home/sn666/.conda/envs/lsdp/lib/python3.11/site-packages/ax/storage/json_store/decoder.py:303: AxParameterWarning: `sort_values` is not specified for `ChoiceParameter` "layer_3_size". Defaulting to `True` for parameters of `ParameterType` INT. To over

4 points: OrderedDict([(1, ({'layer_1_size': 32, 'layer_2_size': 32, 'layer_3_size': 128, 'dropout': 0.13657920844852925, 'batch_size': 128, 'learning_rate': 0.005010103814397345}, ({'ptl/model_params': np.float64(31690.27574056268), 'ptl/val_accuracy': np.float64(0.9536733467608635)}, {'ptl/model_params': {'ptl/model_params': 1274.6268473757307, 'ptl/val_accuracy': 0.0}, 'ptl/val_accuracy': {'ptl/model_params': 0.0, 'ptl/val_accuracy': 6.789935281211314e-05}}))), (24, ({'layer_1_size': 32, 'layer_2_size': 32, 'layer_3_size': 64, 'dropout': 0.14507696150884894, 'batch_size': 128, 'learning_rate': 0.003847365757619393}, ({'ptl/model_params': np.float64(28937.02663647696), 'ptl/val_accuracy': np.float64(0.9498620696508727)}, {'ptl/model_params': {'ptl/model_params': 1981.7756666757844, 'ptl/val_accuracy': 0.0}, 'ptl/val_accuracy': {'ptl/model_params': 0.0, 'ptl/val_accuracy': 4.3573459001461335e-05}}))), (3, ({'layer_1_size': 16, 'layer_2_size': 32, 'layer_3_size': 128, 'dropout': 0.1732

/home/sn666/.conda/envs/lsdp/lib/python3.11/site-packages/ax/storage/json_store/decoder.py:303: AxParameterWarning: `sort_values` is not specified for `ChoiceParameter` "layer_1_size". Defaulting to `True` for parameters of `ParameterType` INT. To override this behavior (or avoid this warning), specify `sort_values` during `ChoiceParameter` construction.
  return _class(
/home/sn666/.conda/envs/lsdp/lib/python3.11/site-packages/ax/storage/json_store/decoder.py:303: AxParameterWarning: `sort_values` is not specified for `ChoiceParameter` "layer_2_size". Defaulting to `True` for parameters of `ParameterType` INT. To override this behavior (or avoid this warning), specify `sort_values` during `ChoiceParameter` construction.
  return _class(
/home/sn666/.conda/envs/lsdp/lib/python3.11/site-packages/ax/storage/json_store/decoder.py:303: AxParameterWarning: `sort_values` is not specified for `ChoiceParameter` "layer_3_size". Defaulting to `True` for parameters of `ParameterType` INT. To over

1 points: OrderedDict([(14, ({'layer_1_size': 32, 'layer_2_size': 64, 'layer_3_size': 128, 'dropout': 0.12276644388045341, 'batch_size': 64, 'learning_rate': 0.0020811463202035597}, ({'ptl/val_accuracy': np.float64(0.9526570827754232), 'ptl/val_loss': np.float64(0.1488865134450113)}, {'ptl/val_accuracy': {'ptl/val_accuracy': 2.1641183986332495e-05, 'ptl/val_loss': 0.0}, 'ptl/val_loss': {'ptl/val_accuracy': 0.0, 'ptl/val_loss': 0.0002505443700168333}})))])
Hypervolume: 0.018068896702381647
Experiment: 1dec9f6b115f408f8538c45981971da0, Best accuracy trial config: {'config/layer_1_size': 32, 'config/layer_2_size': 32, 'config/layer_3_size': 128, 'config/dropout': 0.1832830986008048, 'config/batch_size': 64, 'config/learning_rate': 0.0017525716958184}


In [6]:
relevant_experiments

{'maxaccminlossminparam': '7e8b8fd6db924b6d96d2dbd49ac66243',
 'maxaccminparam': '948b5cdabbef4306b36134190bb536e2',
 'micaccminloss': '7d7908df79254cbc8f2f2d857e48e102',
 'minlossminparam': 'bc6ae72975ca43fab99bf51223d47746'}

In [12]:
exp_path = "/home/sn666/large-scale-data-processing/miniproject/results/fa0fc67004954505a672abce243651f5/tuning_results_df.csv"

df = pd.read_csv(exp_path)

In [13]:
df.columns

Index(['Unnamed: 0', 'ptl/train_loss', 'ptl/train_accuracy', 'ptl/val_loss',
       'ptl/val_accuracy', 'ptl/model_params', 'epoch', 'step', 'timestamp',
       'checkpoint_dir_name', 'should_checkpoint', 'done',
       'training_iteration', 'trial_id', 'date', 'time_this_iter_s',
       'time_total_s', 'pid', 'hostname', 'node_ip', 'time_since_restore',
       'iterations_since_restore', 'config/layer_1_size',
       'config/layer_2_size', 'config/layer_3_size', 'config/dropout',
       'config/batch_size', 'config/learning_rate', 'logdir'],
      dtype='object')

In [14]:
df['time_this_iter_s']

0       1.612234
1       2.416968
2       2.567759
3       1.692087
4       2.044811
5      12.477144
6       2.981579
7      12.379710
8      13.863917
9      12.286338
10     12.264361
11    108.054820
12      1.463883
13      1.454283
14     76.343006
15     81.416552
16    103.267424
17     95.734978
18    108.520681
19      2.095678
20    112.857211
21      1.496494
22     98.935167
23     80.788909
24      2.159046
Name: time_this_iter_s, dtype: float64

In [15]:
df['time_total_s'].sum() / 60

np.float64(24.657567210992177)

In [9]:
df = pd.read_csv("/home/sn666/large-scale-data-processing/miniproject/results/fc3d8e4087824bcf971373218e293bed/tuning_results_df.csv")
df["time_this_iter_s"].sum()

np.float64(948.2891850471497)

In [10]:
df["time_total_s"].sum()

np.float64(1371.190170764923)

In [17]:
df2 = pd.read_csv("/home/sn666/large-scale-data-processing/miniproject/results/bc6ae72975ca43fab99bf51223d47746/tuning_results_df.csv")

df2["time_total_s"].sum() / 60

np.float64(30.636387499173484)